In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("../../data/model_data.csv")

In [3]:
data['청구금액_B0'] = data['청구금액_B0'].apply(lambda x: 1 if x == 0 else 0)

In [4]:
data['청구금액_B0'].value_counts() # 1이 이탈고객

청구금액_B0
0    2323611
1     676389
Name: count, dtype: int64

In [5]:
data

,VIP등급코드,남녀구분코드,연령,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,입회경과개월수_신용,탈회횟수_누적,최종탈회후경과월,탈회횟수_발급6개월이내,...,이용건수_할부_R3M,이용건수_체크_R3M,이용금액_일시불_R3M,이용금액_할부_R3M,이용금액_체크_R3M,이용개월수_일시불_R3M,이용개월수_할부_R3M,이용개월수_체크_R3M,이용금액_업종기준,청구금액_B0
0,_,2,40,0,0,0,166,0,0,0,...,0,0,-151346,1272,0,0,0,0,0,1
1,_,1,30,0,0,0,103,0,0,0,...,0,0,-53212,-751,0,0,0,0,0,1
2,_,2,60,0,0,0,8,0,0,0,...,0,0,-136007,486,0,0,0,0,0,1
3,_,2,30,0,0,0,2,2,18,0,...,0,0,286,-322,0,0,0,0,0,1
4,_,1,40,0,0,0,72,0,0,0,...,0,0,-41710,5646,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,06,1,40,1,1,0,262,1,45,0,...,1,0,5788866,636917,0,3,1,0,6937435,0
2999996,06,1,50,1,1,1,272,0,0,0,...,0,0,4822807,8712,0,3,0,0,4944998,0
2999997,07,1,30,1,1,1,86,1,96,0,...,0,0,4174146,12935,0,3,0,0,4011335,0
2999998,07,1,40,1,1,1,136,1,100,0,...,0,0,6429260,1754,0,3,0,0,8598625,0


In [6]:
def preprocess_feature(data):
    data['VIP등급코드'] = data['VIP등급코드'].str.replace("_","0").astype(int)
    data['청구금액_B0'] = data['청구금액_B0'].apply(lambda x: 1 if x > 0 else x)
    data['청구금액_B0'] = data['청구금액_B0'].apply(lambda x: 1 if x > 0 else x)
    
    # 이용건수 합치기 예시
    data['이용건수_B0M'] = data[['이용건수_일시불_B0M', '이용건수_체크_B0M', '이용건수_할부_B0M']].sum(axis=1)

    # 이용금액 합치기 예시
    data['이용금액_B0M'] = data[['이용금액_일시불_B0M', '이용금액_체크_B0M', '이용금액_할부_B0M']].sum(axis=1)

    # 이용개월수 합치기 예시
    data['이용개월수_R12M'] = data[['이용개월수_일시불_R12M', '이용개월수_할부_R12M', '이용개월수_체크_R12M']].sum(axis=1)

    # 유효카드수 합치기 예시
    data['유효카드수_총합'] = data[['유효카드수_신용체크', '유효카드수_신용', '유효카드수_체크']].sum(axis=1)

    # 최종 이용일자 최신값 선택 (datetime이라 .max()가 정확히 동작함)
    data['최종이용일자_종합'] = data[['최종이용일자_일시불', '최종이용일자_체크', '최종이용일자_할부']].max(axis=1)

    # 필요 없는 원본 컬럼 제거 (존재하는 컬럼만)
    cols_to_drop = [
        '이용건수_일시불_B0M', '이용건수_체크_B0M', '이용건수_할부_B0M',
        '이용건수_일시불_R12M', '이용건수_할부_R12M', '이용건수_체크_R12M',
        '이용건수_일시불_R6M', '이용건수_할부_R6M', '이용건수_체크_R6M',
        '이용건수_일시불_R3M', '이용건수_할부_R3M', '이용건수_체크_R3M',
        '이용금액_일시불_B0M', '이용금액_체크_B0M', '이용금액_할부_B0M',
        '이용금액_일시불_R12M', '이용금액_할부_R12M', '이용금액_체크_R12M',
        '이용금액_일시불_R6M', '이용금액_할부_R6M', '이용금액_체크_R6M',
        '이용금액_일시불_R3M', '이용금액_할부_R3M', '이용금액_체크_R3M',
        '이용개월수_일시불_R12M', '이용개월수_할부_R12M', '이용개월수_체크_R12M',
        '이용개월수_일시불_R6M', '이용개월수_할부_R6M', '이용개월수_체크_R6M',
        '이용개월수_일시불_R3M', '이용개월수_할부_R3M', '이용개월수_체크_R3M',
        '유효카드수_신용체크', '유효카드수_신용', '유효카드수_체크',
        '최종이용일자_일시불', '최종이용일자_체크', '최종이용일자_할부',
        '이용금액_R3M_신용체크','탈회횟수_발급1년이내'
    ]
    drop_cols = [col for col in cols_to_drop if col in data.columns]
    data.drop(columns=drop_cols, inplace=True)

    return data


In [7]:
data = preprocess_feature(data)

In [8]:
data

,VIP등급코드,남녀구분코드,연령,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,입회경과개월수_신용,탈회횟수_누적,최종탈회후경과월,탈회횟수_발급6개월이내,...,최대이용금액_일시불_R12M,최대이용금액_할부_R12M,최대이용금액_체크_R12M,이용금액_업종기준,청구금액_B0,이용건수_B0M,이용금액_B0M,이용개월수_R12M,유효카드수_총합,최종이용일자_종합
0,0,2,40,0,0,0,166,0,0,0,...,0,0,0,0,1,1,0,0,2,20180716
1,0,1,30,0,0,0,103,0,0,0,...,19070,0,0,0,1,-1,0,2,4,20170917
2,0,2,60,0,0,0,8,0,0,0,...,301285,0,0,0,1,0,0,4,0,20180102
3,0,2,30,0,0,0,2,2,18,0,...,0,0,0,0,1,-1,0,0,2,10101
4,0,1,40,0,0,0,72,0,0,0,...,0,0,0,0,1,0,0,0,2,20160408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,6,1,40,1,1,0,262,1,45,0,...,2521359,980459,157548,6937435,0,82,2129608,19,14,20180730
2999996,6,1,50,1,1,1,272,0,0,0,...,3659484,0,196841,4944998,0,47,1605615,16,14,20180728
2999997,7,1,30,1,1,1,86,1,96,0,...,4028389,0,0,4011335,0,53,1163258,12,16,20180705
2999998,7,1,40,1,1,1,136,1,100,0,...,6073370,421386,0,8598625,0,69,2413165,13,16,20180731


In [9]:
# data.to_csv("model_data.csv",encoding='utf-8')

In [10]:
# def split_date_column(data):
#     # 먼저 10101처럼 잘못된 값을 0으로 처리
#     data['최종이용일자_종합'] = data['최종이용일자_종합'].apply(
#         lambda x: 0 if pd.isnull(x) or x == 10101 else x
#     )

#     # 연, 월, 일 나누기
#     data['최종이용_연도'] = data['최종이용일자_종합'].apply(lambda x: int(str(x)[:4]) if x != 0 else 0)
#     data['최종이용_월'] = data['최종이용일자_종합'].apply(lambda x: int(str(x)[4:6]) if x != 0 else 0)
#     data['최종이용_일'] = data['최종이용일자_종합'].apply(lambda x: int(str(x)[6:8]) if x != 0 else 0)

#     # (선택) 원본 컬럼 제거
#     data.drop(columns=['최종이용일자_종합'], inplace=True)

#     return data


In [11]:
# data = split_date_column(data)

In [12]:
data

,VIP등급코드,남녀구분코드,연령,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,입회경과개월수_신용,탈회횟수_누적,최종탈회후경과월,탈회횟수_발급6개월이내,...,최대이용금액_일시불_R12M,최대이용금액_할부_R12M,최대이용금액_체크_R12M,이용금액_업종기준,청구금액_B0,이용건수_B0M,이용금액_B0M,이용개월수_R12M,유효카드수_총합,최종이용일자_종합
0,0,2,40,0,0,0,166,0,0,0,...,0,0,0,0,1,1,0,0,2,20180716
1,0,1,30,0,0,0,103,0,0,0,...,19070,0,0,0,1,-1,0,2,4,20170917
2,0,2,60,0,0,0,8,0,0,0,...,301285,0,0,0,1,0,0,4,0,20180102
3,0,2,30,0,0,0,2,2,18,0,...,0,0,0,0,1,-1,0,0,2,10101
4,0,1,40,0,0,0,72,0,0,0,...,0,0,0,0,1,0,0,0,2,20160408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,6,1,40,1,1,0,262,1,45,0,...,2521359,980459,157548,6937435,0,82,2129608,19,14,20180730
2999996,6,1,50,1,1,1,272,0,0,0,...,3659484,0,196841,4944998,0,47,1605615,16,14,20180728
2999997,7,1,30,1,1,1,86,1,96,0,...,4028389,0,0,4011335,0,53,1163258,12,16,20180705
2999998,7,1,40,1,1,1,136,1,100,0,...,6073370,421386,0,8598625,0,69,2413165,13,16,20180731


In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 상관계수 행렬 계산
corr_matrix = data.corr().abs()  # 절댓값으로 변환

# 임계값 이상인 변수 쌍 찾기 (예: 0.7 이상)
threshold = 0.7
high_corr = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
                         .stack()
                         .reset_index()
                         .rename(columns={0:'corr'}))

high_corr = high_corr[high_corr['corr'] > threshold]

print(high_corr.sort_values(by='corr', ascending=False))


         level_0         level_1      corr
215  이용금액_R3M_신용       이용금액_업종기준  0.925255
218  이용금액_R3M_신용        이용금액_B0M  0.862249
281    이용금액_업종기준        이용금액_B0M  0.847167
290     이용건수_B0M        이용금액_B0M  0.828606
236    이용후경과월_할부  최대이용금액_할부_R12M  0.733307
147      탈회횟수_누적        최종탈회후경과월  0.707223


In [14]:
X = data.drop(columns = '청구금액_B0')
y = data['청구금액_B0']

In [15]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# import pandas as pd

# # VIF 계산
# vif = pd.DataFrame()
# vif["변수명"] = X.columns
# vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# print(vif)


In [16]:
# data.to_csv("data.csv",index=False,encoding='utf-8')

In [ ]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    f1_score, accuracy_score, recall_score, precision_score,
    average_precision_score, roc_auc_score
)
import numpy as np

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 각 지표 저장용 리스트
f1_scores = []
accuracy_scores = []
recall_scores = []
precision_scores = []
auc_pr_scores = []
roc_auc_scores = []

for train_index, val_index in kf.split(X_train):
    X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    model = RandomForestClassifier(random_state=42)
    model.fit(X_tr, y_tr)

    y_val_pred = model.predict(X_val)
    y_val_proba = model.predict_proba(X_val)[:, 1]  # 확률 값 필요함 (AUC용)

    f1_scores.append(f1_score(y_val, y_val_pred))
    accuracy_scores.append(accuracy_score(y_val, y_val_pred))
    recall_scores.append(recall_score(y_val, y_val_pred))
    precision_scores.append(precision_score(y_val, y_val_pred))
    auc_pr_scores.append(average_precision_score(y_val, y_val_proba))
    roc_auc_scores.append(roc_auc_score(y_val, y_val_proba))

# 평균 출력
print("==== K-Fold 평균 성능 (5-Fold) ====")
print(f"F1 Score: {np.mean(f1_scores):.4f}")
print(f"Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Recall: {np.mean(recall_scores):.4f}")
print(f"Precision: {np.mean(precision_scores):.4f}")
print(f"AUC-PR: {np.mean(auc_pr_scores):.4f}")
print(f"ROC-AUC: {np.mean(roc_auc_scores):.4f}")


In [ ]:
# 전체 학습 데이터로 최종 모델 학습
final_model = RandomForestClassifier(random_state=42)
final_model.fit(X_train, y_train)

# 테스트 데이터 예측 및 확률 계산
y_test_pred = final_model.predict(X_test)
y_test_proba = final_model.predict_proba(X_test)[:, 1]

# 테스트 성능 출력
print("\n==== 테스트 데이터 성능 ====")
print(f"F1 Score: {f1_score(y_test, y_test_pred):.4f}")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_test_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred):.4f}")
print(f"AUC-PR: {average_precision_score(y_test, y_test_proba):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_test_proba):.4f}")



==== 테스트 데이터 성능 ====
F1 Score: 0.9023
Accuracy: 0.9562
Recall: 0.8970
Precision: 0.9077
AUC-PR: 0.9589
ROC-AUC: 0.9894


In [ ]:
from sklearn.metrics import precision_recall_curve, recall_score, precision_score, average_precision_score, f1_score, accuracy_score, roc_auc_score
import numpy as np


# 여러 임계값에서 precision, recall, thresholds 계산
precisions, recalls, thresholds = precision_recall_curve(y_test, y_test_proba)

# thresholds 길이는 precisions, recalls 길이보다 1 작으므로, 마지막 값 제외하고 F1 스코어 계산
f1_scores = 2 * (precisions[:-1] * recalls[:-1]) / (precisions[:-1] + recalls[:-1] + 1e-9)

# F1 Score가 최대가 되는 임계값 선택
best_threshold = thresholds[np.argmax(f1_scores)]
print(f"최적의 임계값: {best_threshold:.4f}")

# 최적 임계값으로 클래스 예측
y_test_pred_class = (y_test_proba >= best_threshold).astype(int)

# 성능 지표 출력
recall = recall_score(y_test, y_test_pred_class)
print(f"Test Recall: {recall:.4f}")

precision = precision_score(y_test, y_test_pred_class)
print(f"Test Precision: {precision:.4f}")

aucpr_score = average_precision_score(y_test, y_test_proba)
print(f"AUC-PR: {aucpr_score:.4f}")

print(f"F1 Score  : {f1_score(y_test, y_test_pred_class):.4f}")

accuracy = accuracy_score(y_test, y_test_pred_class)
print(f"Accuracy: {accuracy:.4f}")

roc_auc = roc_auc_score(y_test, y_test_proba)
print(f"ROC-AUC: {roc_auc:.4f}")


최적의 임계값: 0.4292
Test Recall: 0.9241
Test Precision: 0.8860
AUC-PR: 0.9589
F1 Score  : 0.9046
Accuracy: 0.9561
ROC-AUC: 0.9894


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# 예측값과 실제값
cm = confusion_matrix(y_test, y_test_pred_class)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Pred 0', 'Pred 1'], yticklabels=['True 0', 'True 1'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# import joblib
# joblib.dump(model, 'k-fold_42.pkl')

['k-fold_42.pkl']